In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import folium
import datetime

df = pd.read_csv("Police_Department_Incident_Reports__Historical_2003_to_May_2018.csv")
df.head()

,PdId,IncidntNum,Incident Code,Category,Descript,DayOfWeek,Date,Time,PdDistrict,Resolution,...,Fix It Zones as of 2017-11-06 2 2,DELETE - HSOC Zones 2 2,Fix It Zones as of 2018-02-07 2 2,"CBD, BID and GBD Boundaries as of 2017 2 2","Areas of Vulnerability, 2016 2 2",Central Market/Tenderloin Boundary 2 2,Central Market/Tenderloin Boundary Polygon - Updated 2 2,HSOC Zones as of 2018-06-05 2 2,OWED Public Spaces 2 2,Neighborhoods 2
0,4018830907041,40188309,7041,VEHICLE THEFT,"VEHICLE, RECOVERED, AUTO",Sunday,02/15/2004,02:00,SOUTHERN,NONE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,13605159706372,136051597,6372,LARCENY/THEFT,PETTY THEFT OF PROPERTY,Thursday,03/14/2013,09:21,SOUTHERN,NONE,...,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,32.0
2,13618826906244,136188269,6244,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Tuesday,10/08/2013,19:30,MISSION,NONE,...,NaN,3.0,NaN,NaN,2.0,NaN,NaN,3.0,NaN,NaN
3,4098938926105,40989389,26105,OTHER OFFENSES,FALSE REPORT OF BOMB,Sunday,08/29/2004,22:47,PARK,NONE,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,49.0
4,5085058207021,50850582,7021,VEHICLE THEFT,STOLEN AUTOMOBILE,Friday,07/29/2005,21:00,INGLESIDE,NONE,...,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,66.0


In [2]:
from sklearn.preprocessing import OrdinalEncoder

ord_enc = OrdinalEncoder()
df['Date'] = pd.to_datetime(df['Date'])
df["DayOfWeekNum"] = ord_enc.fit_transform(df[["DayOfWeek"]])
df["Month"] = df.Date.apply(lambda x: x.month)
df['Time'] = pd.to_datetime(df['Time'])
df.Time = df.Time.apply(lambda x: x.time())
df["Hour"] = df.Time.apply(lambda x: x.hour)
df["PdDistrictNum"] = ord_enc.fit_transform(df[["PdDistrict"]].fillna(""))

In [3]:
crimes = df.loc[(df.Category == "BURGLARY") | (df.Category == "FRAUD")]
crimes.Category.value_counts()

BURGLARY    91067
FRAUD       41348
Name: Category, dtype: int64

In [27]:
training = pd.concat([crimes.loc[df.Category=="FRAUD"].sample(n=20000), crimes.loc[df.Category=="BURGLARY"].sample(n=20000)], ignore_index=True)

In [28]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(random_state=0)
X = training[["PdDistrictNum", "DayOfWeekNum", "Month", "Hour"]]
training["CategoryNum"] = ord_enc.fit_transform(training[["Category"]])
y = training[["CategoryNum"]]
training["date"] = pd.to_datetime(training.Date.astype(str)+' '+training.Time.astype(str)).round("60min")

In [29]:
from sklearn.model_selection import train_test_split
from sklearn import metrics

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2) # 70% training and 30% test

clf = clf.fit(X_train,y_train)

y_pred = clf.predict(X_test)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.5725


56% Is pretty damn shitty :/
# Weather data

In [30]:
weather = pd.read_csv("weather_data.txt")
weather.date = (pd.to_datetime(weather.date) - pd.Timedelta(hours=7)).dt.tz_localize(None)
weather.head(3)

,date,temperature,humidity,weather,wind_speed,wind_direction,pressure
0,2012-10-01 06:00:00,16.330000,88.0,light rain,2.0,150.0,1009.0
1,2012-10-01 07:00:00,16.324993,87.0,sky is clear,2.0,147.0,1009.0
2,2012-10-01 08:00:00,16.310618,86.0,sky is clear,2.0,141.0,1009.0


In [50]:
merge=pd.merge(training,weather, how='inner', on='date')

In [58]:
merge["weathernum"] = ord_enc.fit_transform(merge[["PdDistrict"]].fillna(""))
X = merge[["PdDistrictNum", "DayOfWeekNum", "Month", "Hour", "weathernum", "wind_direction", "temperature", "wind_speed"]]
y = merge[["CategoryNum"]]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1) # 70% training and 30% test

clf = DecisionTreeClassifier(random_state=0)
clf = clf.fit(X_train,y_train)

y_pred = clf.predict(X_test)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
X

Accuracy: 0.553175775480059


,PdDistrictNum,DayOfWeekNum,Month,Hour,weathernum,wind_direction,temperature,wind_speed
0,4.0,1.0,7,12,3.0,350.0,26.240000,5.0
1,3.0,1.0,7,12,2.0,350.0,26.240000,5.0
2,10.0,0.0,1,21,9.0,303.0,10.790000,1.0
3,8.0,3.0,11,11,7.0,290.0,12.710000,7.0
4,8.0,3.0,11,11,7.0,290.0,12.710000,7.0
...,...,...,...,...,...,...,...,...
13534,9.0,4.0,8,6,8.0,208.0,13.447667,1.0
13535,4.0,2.0,7,21,3.0,207.0,23.380000,1.0
13536,10.0,1.0,8,19,9.0,232.0,18.570000,1.0
13537,1.0,4.0,2,3,0.0,290.0,6.070000,6.0


# Exercises: Explanatory data visualization

* What are the three key elements to keep in mind when you design an explanatory visualization?

Start with a question, allow exploration, know your readers.
* In the video I talk about (1) overview first, (2) zoom and filter, (3) details on demand.
 * Go online and find a visualization that follows these principles (don't use one from the video).
 * Explain how it does achieves (1)-(3). It might be useful to use screenshots to illustrate your explanation.
* Explain in your own words: How is explanatory data analysis different from exploratory data analysis?